Would be nice to be able to open a bunch of neuronpedias side by side, or open a list of them. I think there is a function for this.

- What ideas, concepts and linguistic features have these models learned? In other words, what is the vocabulary and language that we have at our disposal?
    - Need to be able to learn about what different features composed of, some features are better than others for conceptual steering
- What effects do these features have on the output of the model?
    - Add, subtract, add multiple subtract multiple, at different ratios.
- How will we give instructions (method and interface) to edit these features? Will we eventually want to edit groups of related features representing complex concepts all at once?
    - Many interface questions.
    - What meaningful structure exists between features?
        - Feature families.
        - Equivariance.
            - I want to be able to pass arbitrary prompts through a feature to see how it activates.
        - Feature splitting
            - Different levels of abstraction all at once.

In [ ]:
%pip install sae_lens
%pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.0/920.0 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from sae_lens import LMSparseAutoencoderSessionloader
from huggingface_hub import hf_hub_download

In [ ]:
# For the most part I'll try to import functions and classes near where they are used
# to make it clear where they come from.

torch.set_grad_enabled(False)

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes

# let's start with a layer 8 SAE.
hook_point = "blocks.8.hook_resid_pre"

# if the SAEs were stored with precomputed feature sparsities,
#  those will be return in a dictionary as well.
saes, sparsities = get_gpt2_res_jb_saes(hook_point)

print(saes.keys())

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


blocks.8.hook_resid_pre/cfg.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

sae_weights.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

sparsity.safetensors:   0%|          | 0.00/98.4k [00:00<?, ?B/s]

100%|██████████| 1/1 [00:02<00:00,  2.65s/it]

dict_keys(['blocks.8.hook_resid_pre'])


In [ ]:
print(saes['blocks.8.hook_resid_pre'])

SparseAutoencoder(
  (activation_fn): ReLU()
  (hook_sae_in): HookPoint()
  (hook_hidden_pre): HookPoint()
  (hook_hidden_post): HookPoint()
  (hook_sae_out): HookPoint()
)


In [ ]:
from sae_lens.training.session_loader import LMSparseAutoencoderSessionloader

sparse_autoencoder = saes[hook_point]
sparse_autoencoder.to(device)
sparse_autoencoder.cfg.device = device

loader = LMSparseAutoencoderSessionloader(sparse_autoencoder.cfg)

# don't overwrite the sparse autoencoder with the loader's sae (newly initialized)
model, sae_dictionary, activation_store = loader.load_sae_training_group_session()

# TODO: We should have the session loader go straight to huggingface.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# # print()
# for data in activation_store.dataset:
#   print(data)

In [ ]:
def get_top_features(prompt, n_features=10):
  tokens = model.tokenizer.tokenize(prompt)
  _, cache = model.run_with_cache(prompt, prepend_bos=True)

  # Use the SAE
  sae_out, feature_acts, loss, mse_loss, l1_loss, _ = sparse_autoencoder(
      cache[sparse_autoencoder.cfg.hook_point]
  )
  indices = feature_acts[:, 1:].topk(n_features).indices[0]
  values = feature_acts[:, 1:].topk(n_features).values[0]

  data = []

  for i, j, k in zip(indices, values, tokens):
    data.append({"indices": i.tolist(), "values": j.tolist(), "token": k})

  return data

test = get_top_features("John and Mary went to the store.")
print(test)

[{'indices': [22660, 658, 7920, 24371, 4690, 23306, 19249, 6126, 11065, 3254], 'values': [51.2866096496582, 20.979063034057617, 14.724337577819824, 6.486481666564941, 3.3089423179626465, 2.796454906463623, 1.808924674987793, 1.5294626951217651, 1.1234862804412842, 1.0648826360702515], 'token': 'John'}, {'indices': [23115, 2833, 8787, 15229, 23389, 24164, 426, 7451, 8906, 22182], 'values': [35.64717483520508, 14.586932182312012, 9.371505737304688, 7.423129081726074, 5.838878631591797, 4.623136520385742, 4.395914077758789, 4.060179233551025, 3.715121030807495, 3.5542149543762207], 'token': 'Ġand'}, {'indices': [13484, 9695, 23603, 22190, 3488, 1562, 17256, 658, 5926, 9556], 'values': [55.79766845703125, 22.550264358520508, 7.689982891082764, 6.773390769958496, 5.955512046813965, 4.133601665496826, 3.858006477355957, 3.6158840656280518, 3.316751003265381, 3.12091064453125], 'token': 'ĠMary'}, {'indices': [12211, 4675, 20950, 5802, 2010, 13651, 18242, 7451, 11260, 23389], 'values': [41.967

### Get similar features to a specified feature

In [ ]:
import torch.nn.functional as F
# Normalize the embeddings along the embedding dimension
normalized_embeddings = F.normalize(sparse_autoencoder.W_dec, p=2, dim=1)

# Calculate the cosine similarity matrix
cos_sim_matrix = torch.mm(normalized_embeddings, normalized_embeddings.t()).cuda()

print(cos_sim_matrix.shape)

torch.Size([24576, 24576])


In [ ]:
import torch

def find_most_similar(cos_sim_matrix, target_idx, top_k=10, return_rows=True):
    """
    Finds the top-k most similar rows (or columns) to a target row (or column) in a cosine similarity matrix.

    Args:
        cos_sim_matrix (torch.Tensor): The cosine similarity matrix.
        target_idx (int): The index of the target row (or column) to compare against.
        top_k (int, optional): The number of most similar rows (or columns) to return. Default is 10.
        return_rows (bool, optional): If True, return the most similar rows. If False, return the most similar columns. Default is True.

    Returns:
        torch.Tensor: A tensor containing the indices of the top-k most similar rows (or columns).
        torch.Tensor: A tensor containing the similarity scores of the top-k most similar rows (or columns).
    """
    target_vector = cos_sim_matrix[target_idx] if return_rows else cos_sim_matrix[:, target_idx]
    similarities = target_vector if return_rows else target_vector.T

    topk_similarities, topk_indices = torch.topk(similarities, k=top_k, largest=True, sorted=True)

    if not return_rows:
        topk_indices = topk_indices.T

    return topk_indices, topk_similarities

topk_indices, topk_similarities = find_most_similar(cos_sim_matrix, 658)

print(topk_indices)
print(topk_similarities)

tensor([  658,  6521,  6126, 19249, 12937,  4587, 22182, 10125,  6041,  5926],
       device='cuda:0')
tensor([1.0000, 0.6647, 0.6628, 0.6214, 0.6080, 0.6069, 0.5751, 0.5723, 0.5655,
        0.5633], device='cuda:0')


In [ ]:
def get_similar_vocab(token_string):
  vocab_index = model.tokenizer.encode(token_string)[0]

  indices, values = find_most_similar(cos_sim_matrix, vocab_index, top_k=100)

  return model.tokenizer.batch_decode(indices)

vocab = get_similar_vocab("fire")
print(vocab)

['fire', 'fires', 'Fire', ' fire', 'fired', ' Fire', 'flame', ' FIRE', ' fires', ' Fires', ' wildfire', ' flames', ' blaze', ' flame', ' firefighters', 'burning', ' firefighter', ' fireplace', ' wildfires', ' fired', 'flash', ' fireball', ' firing', 'spot', 'FI', ' Flames', 'load', ' arson', ' burner', ' lightning', ' gunfire', ' Fired', 'munition', 'senal', ' ignite', 'hots', 'strike', 'blast', 'iery', 'damage', ' combust', ' firewall', 'burn', 'light', 'storm', ' Flame', 'bolt', ' fireworks', ' furnace', 'Thunder', ' extingu', 'lash', ' Firearms', 'loads', 'grounds', 'erno', ' fiery', 'weapons', ' incendiary', 'flower', 'peace', 'road', 'hot', 'gun', 'dragon', ' flaming', ' Firefly', 'watch', ' burns', 'death', ' Flare', 'blade', 'sight', 'wire', 'oak', 'trigger', 'Storm', 'wolves', 'guns', 'forge', 'hawk', 'casters', ' burning', 'steel', ' Inferno', 'heat', ' ashes', 'lights', 'mine', ' stove', 'spawn', 'nova', 'wolf', ' burned', ' combustion', ' burn', 'earth', 'launch', ' blazing'

In [ ]:
get_top_features("For the love of war")

[{'indices': [18376,
   7920,
   10643,
   10407,
   15684,
   13968,
   1319,
   12082,
   24236,
   22689],
  'values': [43.09342575073242,
   19.27932357788086,
   10.006363868713379,
   9.633081436157227,
   5.505853652954102,
   2.4890196323394775,
   2.3744096755981445,
   1.719088077545166,
   1.0363314151763916,
   0.775364100933075],
  'token': 'For'},
 {'indices': [9691, 1227, 495, 9257, 7920, 10407, 23217, 7097, 9237, 3008],
  'values': [26.826580047607422,
   24.719228744506836,
   9.594332695007324,
   7.852558135986328,
   7.554095268249512,
   5.348336696624756,
   5.137289524078369,
   3.8822529315948486,
   3.2913308143615723,
   2.668774127960205],
  'token': 'Ġthe'},
 {'indices': [14635,
   23217,
   21121,
   19569,
   16021,
   11744,
   1932,
   6160,
   15075,
   7517],
  'values': [48.7425651550293,
   14.197075843811035,
   6.711040019989014,
   6.496437072753906,
   6.385776996612549,
   5.855486869812012,
   5.655556678771973,
   5.603943824768066,
   5.29699

Need an interface here to view and then select a feature(s) to control.


I want to view features firing for a particular token. I want to manipulate the amount those features are firing. Remove features. Replace them with other ones. Then continue the process.

Problem, feature activations are token based. I would need a way to find similar elements in the embedding vocab space to a token.

Can use similar tokens in the embedding space to move laterally in SAE feature space, finding features not represented previously that could be reasonable in that moment

In [ ]:
feature_out = sparse_autoencoder.W_dec[8713]

print(feature_out.shape)
print(feature_out.sum().item())

torch.Size([768])
-1.2024484872817993


In [ ]:
print(sparse_autoencoder)

SparseAutoencoder(
  (activation_fn): ReLU()
  (hook_sae_in): HookPoint()
  (hook_hidden_pre): HookPoint()
  (hook_hidden_post): HookPoint()
  (hook_sae_out): HookPoint()
)


In [ ]:
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from jaxtyping import Float
from functools import partial
import random

### Places to intervene

- Number of tokens to steer for
- Layer(s) to steer on
- Coefficient to multiply by
- Should I subtract an equal and opposite SAE feature?


In [ ]:
STEER_FOR = 25

def intervention_hook(
    value: Float[torch.Tensor, "batch pos d_model"],
    hook: HookPoint,
    feature_vectors: Float[torch.Tensor, "num d_model"],
    activations: Float[torch.Tensor, "num"],
    intensity: float
) -> Float[torch.Tensor, "batch pos d_head"]:
    # add all feature_vectors to vector
    for i, feature_vector in enumerate(feature_vectors):
      value[:, 4, :] += feature_vector * activations[i] * intensity
    return value

def get_feature_vectors(features=[]):
  feature_vectors = torch.zeros(len(features), model.cfg.d_model).to(device)
  for i, feature in enumerate(features):
    feature_vectors[i] = sparse_autoencoder.W_dec[feature].to(device)
  return feature_vectors

def run_intervention(prompt, feature_list=[], intensity = 10):
  # [[1,2], ...] unzip features list
  features = [feature[0] for feature in feature_list]
  activations = [feature[1] for feature in feature_list]

  feature_vectors = get_feature_vectors(features)
  # print(f"before: {prompt}")

  # print(model.tokenizer.tokenize(prompt))

  temp_hook_fn = partial(intervention_hook, feature_vectors=feature_vectors, activations=activations, intensity=intensity)
  for i in range(50):
    if i < STEER_FOR:
      logits = model.run_with_hooks(
        prompt,
        fwd_hooks=[(
            sparse_autoencoder.cfg.hook_point,
            temp_hook_fn
            )]
        )
    else:
      logits, _ = model.run_with_cache(prompt)
    # prompt += model.tokenizer.decode(logits[0][-1].argmax())

    # print(logits[0][-1].topk(10))
    prompt += model.tokenizer.decode(random.choice(logits[0][-1].topk(3).indices))
  print(f"{prompt}")

# for i in range(3):
#   run_intervention("It was a quiet afternoon")
#   print("\n")

# print("\n\n")

# for i in range(3):
#   run_intervention("It was a quiet afternoon", [[14635, 10], [17085, 10]])
#   print("\n")


In [ ]:
feature_lists = get_top_features("writer")

print(feature_lists)


feature_pairs = []
for feature_list in feature_lists:
  # zip the indices and values in feature list
  for feature, activation in zip(feature_list["indices"], feature_list["values"]):
    if activation > 10:
      feature_pairs.append([feature, activation])

feature_pairs = [feature_pairs[0]]
print(feature_pairs)

[{'indices': [2443, 13586, 16346, 4286, 12741, 1064, 14928, 987, 443, 19807], 'values': [27.357994079589844, 15.404810905456543, 10.413387298583984, 8.519888877868652, 7.548379421234131, 7.184719085693359, 4.865588665008545, 3.740849494934082, 3.684185743331909, 3.5564582347869873], 'token': 'writer'}]
[[2443, 27.357994079589844]]


In [ ]:
travel = [18904, 42.82036209106445]
love = [14635, 42.59407424926758]
food = [5234, 46.90765380859375]
writer = [2443, 27.357994079589844]

feature_pairs = [writer]

# for i in range(3):
#   run_intervention("It was a quiet afternoon")
#   print("\n")

print("\n\n")

print(feature_pairs)

for i in range(3):
  run_intervention("It was a quiet afternoon", feature_pairs, 30)
  print("\n")




[[2443, 27.357994079589844]]
It was a quiet afternoon, but it wasn't until I went to the gym that my body started writing. I was a bit of writing in the middle, and I didn

, I didn`ll be writing for a long while. But I was doing it.


It was a quiet afternoon at my house. It had become an afternoon of writing, of watching my house burn down, and I was going to be writing a story about the death of the family that had once been my family, my wife. I'd written it about the


It was a quiet afternoon, and I had been out of town all writing for the writing school for the last year. I had a lot more than just my writing classes, but I also got to meet some of the best people I've ever known, including a few that




  feature_pairs = [writer]
  run_intervention("It was a quiet afternoon", feature_pairs, 100)

  this was intervening on 0:5 token positions

POSITIVE LOGITS

writing
0.99
writer
0.95
writ
0.93
 laureate
0.92
uscript
0.87
 fiction
0.85
 Beware
0.84
writers
0.83
pub
0.79
 writer
0.7

It was a quiet afternoon writers writers writers writer writing writer writing writer writer writer writers writing writer writers writing writer writer writer writer writing writer writers writers writing writers writers writers writer writing writers writers writer writers writer writer writing writer writer writing writing writing writers writer writer writing writer writer writers writing writing


It was a quiet afternoon writers writers writing writer writing writer writers writer writers writers writing writer writing writers writing writing writing writers writing writers writers writers writers writing writers writers writing writing writing writer writer writer writing writers writer writing writers writing writer writers writer writing writer writer writing writers writer writing writing writing


It was a quiet afternoon writing writer writing writers writers writing writing writing writers writing writers writing writer writers writers writing writing writer writer writing writers writers writing writer writers Writing Writer Writer Writer Writing Writer Writer writing writers writers Writing Writer writing writing writer Writing Writing Writer writing writers Writing Writer Writing Writer Writing



In [ ]:
logits, cache = model.run_with_cache("Wedding")

model.tokenizer.tokenize("Wedding")

['W', 'edd', 'ing']

In [ ]:
print(cache['blocks.8.hook_resid_pre'])

tensor([[[-5.0307, -5.0059, -4.4554,  ..., -3.5955, -3.8039, -4.3690],
         [-3.0007,  2.0551, -1.3563,  ..., -8.7921, -0.1540,  0.4602],
         [ 0.8923,  1.7546, -1.6637,  ..., -0.1999,  2.2241,  2.0386],
         [ 1.8643, -0.8348,  1.0155,  ...,  4.2159, -0.8007, -1.2970]]],
       device='cuda:0')


In [ ]:
TARGET_LAYER = 8

def get_steering_vector(prompt, layer, coefficient):
  logits, cache = model.run_with_cache(prompt)
  layer_activations = cache[f"blocks.{layer}.hook_resid_pre"]
  steering_vectors = layer_activations * coefficient

  ## remove batch
  return steering_vectors[0]

print(get_steering_vector("John and Hailey went to the", TARGET_LAYER, 100))

tensor([[-503.0691, -500.5889, -445.5382,  ..., -359.5479, -380.3918,
         -436.9012],
        [-289.1674,  432.8777,  -40.5833,  ...,  169.0353,    5.4933,
         -133.5507],
        [ 299.1373, -389.4969,   21.9871,  ..., -261.5161,  313.9923,
           -5.3438],
        ...,
        [-150.7217, -412.2509,  187.9374,  ...,   25.7218, -191.8268,
          -41.8348],
        [ 241.9812, -356.3047,  113.4736,  ..., -396.9736,  -87.2059,
           48.2342],
        [  28.5529,  -30.2636, -111.9411,  ...,  -95.7274,  -63.9478,
           34.1887]], device='cuda:0')


In [ ]:
sparse_autoencoder.cfg.hook_point

'blocks.8.hook_resid_pre'

## Steering vectors

Note: You can intervene for a few tokens to shift the direction of the text, then return to normal model generation

In [ ]:
import random
TARGET_LAYER = 8
RUNS = 5
INTERVENE_FOR = 50

def add_steering_vector_hook(
    value: Float[torch.Tensor, "batch pos d_model"],
    hook: HookPoint,
    steering_vectors: Float[torch.Tensor, "pos d_model"],
) -> Float[torch.Tensor, "batch pos d_head"]:
    for i, steering_vector in enumerate(steering_vectors):
      value[0, 0:steering_vector.shape[0], :] += steering_vector
    return value

def apply_steering_vector(prompt, layer, coefficient):
  logits, cache = model.run_with_cache(prompt)
  layer_activations = cache[f"blocks.{layer}.hook_resid_pre"]
  steering_vectors = layer_activations * coefficient
  return steering_vectors[0]

def run_steering_intervention(prompt, steering_vectors=[]):
  temp_hook_fn = partial(add_steering_vector_hook, steering_vectors=steering_vectors)
  for i in range(50):
    if i < INTERVENE_FOR:
      logits = model.run_with_hooks(
        prompt,

        fwd_hooks=[(
            f"blocks.{TARGET_LAYER}.hook_resid_pre",
            temp_hook_fn
            )]
        )
    else:
      logits, _ = model.run_with_cache(prompt)
    # prompt += model.
    # print(logits.shape)
    # prompt += model.tokenizer.decode(logits[0][-1].argmax(dim=-1))
    prompt += model.tokenizer.decode(random.choice(logits[0][-1].topk(3).indices))
  print(f"after: {prompt}")







RUN: 0

after: It was a quiet afternoon. The rain was thick, the wind sheathed her hair in a rich, wet-soverere, she felt herself to have been struck with a pleasant, pleasant and happy experience, her life in her very happy, very fortunate happiness.




RUN: 1

after: It was a quiet afternoon in the middle-of the evening of July 14th. The night was very different to the day before. It is the first of the happiest and happiest years in the world's history and the first of our own. The happiest of the happiest.



RUN: 2

after: It was a quiet afternoon, but I felt the heat in me. The sun had just set.
I could see the heat in my own eyes, and I could feel it. I felt the love that shererevery.
Sheresa, I can say



RUN: 3

after: It was a quiet afternoon in the city of Bishwash. The sun has turned into a beautiful evening, with the sun's rays making a happy happy, and then it's time for some serious business. The happy happy, the happiest.


Bin



RUN: 4

after: It was a quiet after

In [ ]:
for i in range(RUNS):
  print(f"\n\n\nRUN: {i}\n")
  run_steering_intervention("It was a quiet afternoon", [
      get_steering_vector("Peace", TARGET_LAYER, 5),
      get_steering_vector("War", TARGET_LAYER, -5),
      get_steering_vector("Love", TARGET_LAYER, 5),
      get_steering_vector("Hate", TARGET_LAYER, -5),
  ])

In [ ]:
for i in range(RUNS):
  print(f"\n\n\nRUN: {i}\n")
  run_steering_intervention("It was a quiet afternoon", [
      get_steering_vector("Writer", TARGET_LAYER, 5),
      get_steering_vector("Speaker", TARGET_LAYER, -5),
  ])




RUN: 0

after: It was a quiet afternoon in a tiny town in a sleepy town of three-year-and-a. I was there to pick up the story of a young man and his friends who had just finished reading a well written, and highly entertaining, storyteller named,



RUN: 1

after: It was a quiet afternoon at my home in New Haven, Connecticut and the sound was familiar to many. I was in the middle of writing an essay about a local high-pitch-and-booth singer named John Hughes and his bandmate, singer/song writer



RUN: 2

after: It was a quiet afternoon in a tiny town called Little Rock's former home. A man, named John, was in a nearby barbershop. He was a tall and imposing figure with long, long, long-haired manacment. He had been living in a



RUN: 3

after: It was a quiet afternoon, a few hours of writing, a handful, one night, and then I heard her say the words that I knew, and the sound of a man in her bed, and I knew it was her voice that I could not help.
.



RUN: 4

after: It was a quiet a

Experiment to see if I could get the closest matrices to this combined feature.

In [ ]:
feature_vectors = get_feature_vectors([8713, 2])
print(feature_vectors[0].shape)
print(feature_vectors[1].shape)
combined_features = feature_vectors.sum(dim=0).cuda()
# print(combined_features)

# Normalize the target vector
target_vector = F.normalize(combined_features, p=2, dim=0)

# print(target_vector)

# Calculate cosine similarity between target vector and each row of cos_sim_matrix
similarities = torch.mm(combined_features.view(1, -1), normalized_embeddings.t().cuda())

print(similarities.shape)

print(similarities.topk(10))

# Get the indices of rows sorted by similarity
sorted_indices = torch.argsort(similarities, descending=True).cuda()

print(sorted_indices)

# Extract the rows from the cosine similarity matrix
most_similar_rows = cos_sim_matrix[sorted_indices]

print(most_similar_rows.shape)

torch.Size([768])
torch.Size([768])
torch.Size([1, 24576])
torch.return_types.topk(
values=tensor([[0.9835, 0.9835, 0.6534, 0.6001, 0.5836, 0.5507, 0.5189, 0.5035, 0.4914,
         0.4864]], device='cuda:0'),
indices=tensor([[    2,  8713,  6933,  5228, 20917, 21147, 12740, 12165, 10128, 15539]],
       device='cuda:0'))
tensor([[    2,  8713,  6933,  ...,  9557, 23506,  8236]], device='cuda:0')
torch.Size([1, 24576, 24576])


In [ ]:
# from tqdm.auto import tqdm


# def get_tokens(
#     activation_store,
#     n_batches_to_sample_from: int = 2**13,
#     n_prompts_to_select: int = 4096 * 6,
# ):
#     all_tokens_list = []
#     pbar = tqdm(range(n_batches_to_sample_from))
#     for _ in pbar:
#         batch_tokens = activation_store.get_batch_tokens()
#         batch_tokens = batch_tokens[torch.randperm(batch_tokens.shape[0])][
#             : batch_tokens.shape[0]
#         ]
#         all_tokens_list.append(batch_tokens)

#     all_tokens = torch.cat(all_tokens_list, dim=0)
#     all_tokens = all_tokens[torch.randperm(all_tokens.shape[0])]
#     return all_tokens[:n_prompts_to_select]


# all_tokens = get_tokens(activation_store)  # should take a few minutes

# # all_tokens = torch.load("all_tokens.pt")

In [ ]:
# from sae_vis.data_config_classes import SaeVisConfig
# from sae_vis.data_storing_fns import SaeVisData

# test_feature_idx_gpt = [24942, 41215, 14553, 16889, 39324, 50601, 43665, 39512,  6939,  5935]

# features = []


# feature_vis_config_gpt = SaeVisConfig(
#     hook_point=sparse_autoencoder.cfg.hook_point,
#     features=test_feature_idx_gpt,
#     batch_size=1024,
#     minibatch_size_tokens=128,
#     verbose=True,
# )

# with torch.inference_mode():
#     sae_vis_data_gpt = SaeVisData.create(
#         encoder=sparse_autoencoder,
#         model=model,
#         tokens=all_tokens,  # type: ignore
#         cfg=feature_vis_config_gpt,
#     )

In [ ]:
# sae_vis_data_gpt.feature_data_dict[1]

In [ ]:
# import os
# from IPython.display import FileLink

# for idx, feature in enumerate(test_feature_idx_gpt):
#     if sae_vis_data_gpt.feature_stats.max[idx] == 0:
#         continue
#     filename = f"{feature}_feature_vis_demo_gpt.html"
#     sae_vis_data_gpt.save_feature_centric_vis(filename, feature)
#     display(FileLink(filename))